In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Note that this must be in its own cell

<IPython.core.display.Javascript object>

In [2]:
# External imports
import ipywidgets as ipw
#from jinja2 import Environment
#from importlib_resources import files

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import BasicStructureEditor
from aiidalab_widgets_base import WizardAppWidget
#from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

In [3]:
from aiidalab_ispg.process import WorkChainSelector
from aiidalab_ispg.steps import SubmitOrcaAppWorkChainStep
from aiidalab_ispg.structures import StructureSelectionStep
from aiidalab_ispg.steps import ViewOrcaAppWorkChainStatusAndResultsStep
from aiidalab_ispg.steps import ViewSpectrumStep

# DH: Our own hacked molecule viewer
from aiidalab_ispg.widget_viewers.viewers import StructureDataViewer

# DH This is for later for importing Jinja templates and styles
#from aiidalab_ispg import static

structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="Upload file"),
        SmilesWidget(title='SMILES'),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    node_class='StructureData',
    viewer=StructureDataViewer(downloadable=False,storable=False),
#    editors = [
#        BasicStructureEditor(title="Basic Editor"),
#    ],
)

structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True

submit_orca_work_chain_step = SubmitOrcaAppWorkChainStep()
submit_orca_work_chain_step.auto_advance = True

view_orca_work_chain_status_and_results_step = ViewOrcaAppWorkChainStatusAndResultsStep()
view_orca_work_chain_status_and_results_step.auto_advance = True

view_spectrum_step = ViewSpectrumStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_orca_work_chain_step, 'input_structure'))
ipw.dlink((submit_orca_work_chain_step, 'process'), (view_orca_work_chain_status_and_results_step, 'process'))
ipw.dlink((view_orca_work_chain_status_and_results_step, 'process'), (view_spectrum_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_orca_work_chain_step),
        ('Status & Results', view_orca_work_chain_status_and_results_step),
        ('Spectrum', view_spectrum_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))
def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_orca_work_chain_step.process = process

work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_orca_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)


app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

display(app_with_work_chain_selector)

Loading BokehJS ...